In [1]:
file_path = '/kaggle/input/dialogs/dialogs.txt'
with open(file_path, 'r') as file:
    # Read the content of the file
    content = file.read()

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer()
tokenizer.fit_on_texts([content])

2024-08-26 14:49:09.840103: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 14:49:09.840233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 14:49:10.008362: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
len(tokenizer.word_index)

2519

In [4]:
input_sequences = []
for sentence in content.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])


In [5]:
max_len = max([len(x) for x in input_sequences])

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [7]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [8]:

from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=len(tokenizer.word_index)+1)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [10]:
len(tokenizer.word_index)+1

2520

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

# Embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=100, 
                    input_length=X.shape[1]))

# First LSTM layer with return_sequences=True
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.3))  # Add Dropout to prevent overfitting

# Second LSTM layer with return_sequences=True
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.3))  # Add Dropout to prevent overfitting

# Third LSTM layer with return_sequences=True
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.3))  # Add Dropout to prevent overfitting

# Fourth LSTM layer without return_sequences
model.add(LSTM(150))
model.add(Dropout(0.3))  # Add Dropout to prevent overfitting

# Dense layer to reduce dimensions before output
model.add(Dense(200, activation='relu'))
model.add(BatchNormalization())  # Add Batch Normalization for stability

# Output layer
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define callbacks
callbacks = [
    EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),  # Stop if no improvement in 'loss'
    ModelCheckpoint(filepath='best_model.keras', monitor='loss', save_best_only=True)  # Save the best model
]

model.build(input_shape=(None, X.shape[1]))
model.summary()


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 31, 100)        │       252,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 31, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 31, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 31, 150)        │       180,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 31, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 31, 150)        │       180,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 31, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 150)            │       180,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │        30,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 200)            │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2520)           │       506,520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,481,920 (5.65 MB)

 Trainable params: 1,481,520 (5.65 MB)

 Non-trainable params: 400 (1.56 KB)

In [12]:
model.fit(X, y, epochs=50, batch_size=32, callbacks=callbacks)


Epoch 1/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 24s 12ms/step - accuracy: 0.0244 - loss: 6.8039
Epoch 2/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0346 - loss: 6.2159
Epoch 3/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0335 - loss: 6.1715
Epoch 4/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0349 - loss: 6.1640
Epoch 5/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0381 - loss: 6.1644
Epoch 6/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0340 - loss: 6.1623
Epoch 7/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0343 - loss: 6.1380
Epoch 8/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0349 - loss: 6.1317
Epoch 9/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - accuracy: 0.0387 - loss: 6.1491
Epoch 10/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - accuracy: 0.0375 - loss: 6.1397
Epoch 11/50
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 17s 12ms/step - accuracy: 0.0344 - loss: 6.1452
Epoch 12

In [13]:
import time
import numpy as np
text = "school"

for i in range(5):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=max_len, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)
      time.sleep(1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
school do
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
school do you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
school do you mean
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
school do you mean that
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
school do you mean that i
